# Fixed initial condition

    In this script, we will fix the initial condition of the simulation as the ones that shows the image below. This way we will observe if the system follows the transfer of charge rule predicted in the publication: 

<img src="InitialConfig.png" alt="drawing" width="50%"/>